In [30]:
from dask.distributed import Client
from dask_cloudprovider.aws import FargateCluster

tags = {"project-pa-number": "21691-H2001", "project": "pvinsight"}
cluster = FargateCluster(tags=tags, image="jjcrush/dask:latest")
client = Client(cluster)

d:\Anaconda3\envs\solar-data-tools\lib\contextlib.py:142: UserWarning: Creating your cluster is taking a surprisingly long time. This is likely due to pending resources on AWS. Hang tight! 
  next(self.gen)


In [31]:
cluster.scale(12)

In [32]:
client.get_versions(check=True)

{'scheduler': {'host': {'python': '3.10.13.final.0',
   'python-bits': 64,
   'OS': 'Linux',
   'OS-release': '5.10.198-187.748.amzn2.x86_64',
   'machine': 'x86_64',
   'processor': '',
   'byteorder': 'little',
   'LC_ALL': 'None',
   'LANG': 'C.UTF-8'},
  'packages': {'python': '3.10.13.final.0',
   'dask': '2023.11.0',
   'distributed': '2023.11.0',
   'msgpack': '1.0.7',
   'cloudpickle': '3.0.0',
   'tornado': '6.3.3',
   'toolz': '0.12.0',
   'numpy': '1.26.2',
   'pandas': '2.1.3',
   'lz4': '4.3.2'}},
 'workers': {},
 'client': {'host': {'python': '3.10.13.final.0',
   'python-bits': 64,
   'OS': 'Windows',
   'OS-release': '10',
   'machine': 'AMD64',
   'processor': 'AMD64 Family 25 Model 80 Stepping 0, AuthenticAMD',
   'byteorder': 'little',
   'LC_ALL': 'None',
   'LANG': 'None'},
  'packages': {'python': '3.10.13.final.0',
   'dask': '2023.11.0',
   'distributed': '2023.11.0',
   'msgpack': '1.0.7',
   'cloudpickle': '3.0.0',
   'tornado': '6.3.3',
   'toolz': '0.12.0',


In [33]:
%load_ext autoreload
%autoreload 2
from solardatatools import DataHandler
from solardatatools.dataio import get_pvdaq_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
df = get_pvdaq_data(sysid=34, year=2011, api_key='DEMO_KEY')

[============================================================] 100.0% ...queries complete in 1.5 seconds       



In [35]:
dh = DataHandler(df)
dh.run_pipeline(power_col='ac_power')

d:\Anaconda3\envs\solar-data-tools\lib\site-packages\solardatatools\time_axis_manipulation.py:146: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if avg_day[0] >= thresh:
d:\Anaconda3\envs\solar-data-tools\lib\site-packages\solardatatools\time_axis_manipulation.py:152: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if avg_day[-1] >= thresh:


total time: 13.36 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              1.54s
Cleaning                   0.07s
Filtering/Summarizing      11.74s
    Data quality           0.05s
    Clear day detect       0.09s
    Clipping detect        9.86s
    Capacity change detect 1.75s



In [36]:
import dask

@dask.delayed
def run_pipeline(df):
    dh = DataHandler(df)
    dh.run_pipeline(power_col='ac_power')
    return dh.report(return_values=True)

In [37]:
results = []
for _ in range(3):
    results.append(run_pipeline(df))
dask.compute(results)

([{'length': 1.0,
   'capacity': 109.3,
   'sampling': 15,
   'quality score': 0.9835616438356164,
   'clearness score': 0.4986301369863014,
   'inverter clipping': True,
   'clipped fraction': 0.01643835616438356,
   'capacity change': False,
   'data quality warning': False,
   'time shift correction': False,
   'time zone correction': 0},
  {'length': 1.0,
   'capacity': 109.3,
   'sampling': 15,
   'quality score': 0.9835616438356164,
   'clearness score': 0.4986301369863014,
   'inverter clipping': True,
   'clipped fraction': 0.01643835616438356,
   'capacity change': False,
   'data quality warning': False,
   'time shift correction': False,
   'time zone correction': 0},
  {'length': 1.0,
   'capacity': 109.3,
   'sampling': 15,
   'quality score': 0.9835616438356164,
   'clearness score': 0.4986301369863014,
   'inverter clipping': True,
   'clipped fraction': 0.01643835616438356,
   'capacity change': False,
   'data quality warning': False,
   'time shift correction': False,

2023-11-28 19:43:21,830 - tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x000001A01CEB1750>>, <Task finished name='Task-14260' coro=<SpecCluster._correct_state_internal() done, defined at d:\Anaconda3\envs\solar-data-tools\lib\site-packages\distributed\deploy\spec.py:346> exception=RuntimeError('Worker failed to start')>)
Traceback (most recent call last):
  File "d:\Anaconda3\envs\solar-data-tools\lib\site-packages\tornado\ioloop.py", line 738, in _run_callback
    ret = callback()
  File "d:\Anaconda3\envs\solar-data-tools\lib\site-packages\tornado\ioloop.py", line 762, in _discard_future_result
    future.result()
  File "d:\Anaconda3\envs\solar-data-tools\lib\site-packages\distributed\deploy\spec.py", line 390, in _correct_state_internal
    await asyncio.gather(*worker_futs)
  File "d:\Anaconda3\envs\solar-data-tools\lib\asyncio\tasks.py", line 650, in _wrap_a